In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import bisect

In [26]:
# Warm up: European put option

r, T, K, S0 = 0.05, 0.2, 120, 100
# Define option price
def I(sigma):
    wstar = (np.log(K/S0) - (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    return np.exp(-r*T) * K * norm.cdf(wstar) - S0 * norm.cdf(wstar - sigma * np.sqrt(T))

# Calculate the "true" implied volatility
I_market = 22
sigma_implied = bisect(lambda sigma: I(sigma) - I_market, 0.01, 10)
print(sigma_implied)

0.508372926009589


In [59]:
# Robbins Monroe Algorithm
def Jhat(sigma, N):
    W_T = norm.rvs(loc=0, scale=T, size=N)
    S_T = S0 * np.exp((r - sigma**2 / 2) * T + sigma * W_T)
    f = np.exp(-r*T) * (K - S_T)
    f[f < 0] = 0
    return np.mean(f) - I_market

def RM(rho=0.8, N=100):
    a0 = 2 / (K + S0)
    n = 0
    sigma_cur = 1
    while True:
        alpha_n = a0 / n**rho if n > 0 else a0
        sigma_next = sigma_cur - alpha_n * Jhat(sigma_cur, N)
        if abs(sigma_next - sigma_cur) < 10**(-9):
            break
        sigma_cur = sigma_next
        n+=1
    return sigma_cur

RM()

0.6253363366757174